In [1]:
!pip install langchain
!pip install huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 26.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: jsonpatch
    Found existing installation: jsonpatch 1.32
    Uninstalling jsonpatch-1.32:
      Successfully uninstalled jsonpatch-1.32


In [2]:
import langchain
from langchain import FewShotPromptTemplate, PromptTemplate, HuggingFaceHub
from langchain.memory import ConversationBufferMemory
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
import pandas as pd
import getpass
import os

In [3]:
inference_api_key = getpass.getpass("Enter your HF Inference API Key:\n\n")


Enter your HF Inference API Key:

 ·····································


In [4]:
os.environ['HUGGINGFACEHUB_API_TOKEN']=inference_api_key

In [5]:
llm = HuggingFaceHub(repo_id ='mistralai/Mistral-7B-Instruct-v0.1')

In [14]:
data=pd.read_csv("/kaggle/input/products/products_database.csv",header=0)

subcategory_counts = data['product_sub_catagory'].value_counts()
print(len(subcategory_counts))
sorted_subcategories = subcategory_counts.index

top_subcategories = list(sorted_subcategories[:1900])
top_subcategories=",".join(top_subcategories)

del data,subcategory_counts, sorted_subcategories

2390


In [15]:
print(top_subcategories)

Necklaces & Chains,Bras,Jewellery,T-Shirts,Cases & Covers,Bangles, Bracelets & Armlets,Shirts,Car Interior,Wrist Watches,Dresses,Tops,Casual Shoes,Women's Footwear,Combos,Plant Containers & Sets,Coffee Mugs,Geometry & Pencil Boxes,Bed Linen,Routers,Stickers,Leggings & Jeggings,Ethnic,Kurtis,USB Gadgets,Curtains & Accessories,Showpieces,Formal Shirts,Brooches,Wall Decals & Stickers,Bath Linen,Woks & Kadhais,Pencil Boxes,Batteries,Sweatshirts,Furniture,Cushions, Pillows & Covers,Sweaters,Jeans,Blouse Material,Combos and Kits,Car Sun Shades,Wall Clocks,Ties,Skirts,Religious Idols,Pizza Cutters,Living Room,Backpacks,Lens Hoods,Formal Shoes,Series Lights,Casual & Party Wear Shirts,Polos & T-Shirts,Car Spare Parts,Ballerinas,Car Mobile Accessories,Heels,Sunscreen,PRINT SHAPES Computer Accessories Combos,Men's Clothing,Bulbs,Sunglasses,Screen Protectors,Sports Shoes,Punches & Punching Machines,Rings,Office Supplies,Vanity Boxes,Vehicle Mirrors,Tunics,Candy House T-Shirts,Curtains,Switches & A

In [16]:
examples = [
    {
        "question":"What are some products I should buy if I'm already buying shoes",
        "answer":'''
        Socks,Shoe Polish,Insoles
        '''
    },
    {
        "question":"What are some products I should buy if I'm already buying laptops",
        "answer":'''
        Mouse,Laptop Case,Keyboard
        '''
    },
    {
        "question":"What are some products I should buy if I'm already buying school bags",
        "answer":'''
        Books,Pens,Pencils
        '''
    },
    {
        "question":"What are some products I should buy if I'm already buying beds",
        "answer":'''
        Blankets,Pillows,Bedsheet
        '''
    },
    {
        "question":"What are some products I should buy if I'm already buying toys",
        "answer":'''
        Chocolates,Arts and Craft,Board Games
        '''
    }
]

In [17]:
 example_template = """
User: {question}
AI: {answer}
"""
example_prompt = PromptTemplate(
    input_variables=["question", "answer"],
    template=example_template
)

prefix = """ The assisstant is a world class recommender of products. The assisstant gives 3 most appropriate items most compatible with product in question
Here are some examples of excerpts between user and the assiatant. Give only 3 products as the output from: """+top_subcategories

suffix = """
User: {question}
AI: """

few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["question"],
    example_separator="\n\n"
)

memory = ConversationBufferMemory(
    memory_key='chat_history',
    return_messages=True
)

In [18]:
chain = LLMChain(llm=llm, prompt=few_shot_prompt_template,verbose=True,memory=memory)

In [11]:
q="Tops"

In [12]:
#chain.predict(question="What are 5 products I should buy if I'm already buying "+q + "ensure recommendations are present in "+ top_subcategories)

In [19]:
#This format seems the best
chain.predict(question="What are 3 unique products I should buy with "+q)



> Entering new LLMChain chain...
Prompt after formatting:
 The assisstant is a world class recommender of products. The assisstant gives 3 most appropriate items most compatible with product in question
Here are some examples of excerpts between user and the assiatant. Give only 3 products as the output from: Necklaces & Chains,Bras,Jewellery,T-Shirts,Cases & Covers,Bangles, Bracelets & Armlets,Shirts,Car Interior,Wrist Watches,Dresses,Tops,Casual Shoes,Women's Footwear,Combos,Plant Containers & Sets,Coffee Mugs,Geometry & Pencil Boxes,Bed Linen,Routers,Stickers,Leggings & Jeggings,Ethnic,Kurtis,USB Gadgets,Curtains & Accessories,Showpieces,Formal Shirts,Brooches,Wall Decals & Stickers,Bath Linen,Woks & Kadhais,Pencil Boxes,Batteries,Sweatshirts,Furniture,Cushions, Pillows & Covers,Sweaters,Jeans,Blouse Material,Combos and Kits,Car Sun Shades,Wall Clocks,Ties,Skirts,Religious Idols,Pizza Cutters,Living Room,Backpacks,Lens Hoods,Formal Shoes,Series Lights,Casual & Party Wear Shirts,Po

'\n        T-Shirt,Sweater,Jacket\n        \n\n\n\nUser'

In [ ]:
chain.predict(question="List 3 unique product subcategories to buy with "+q)

In [ ]:
chain.predict(question="What are 3 unique products I should buy with "+"Car Interior")